# Check 2s

Check 2s speeds from GP3S.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of GP3S Query and is distributed under the terms of the GNU General Public License.

GP3S Query is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GP3S Query is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GP3S Query. If not, see https://www.gnu.org/licenses/.

## Import Common Modules

In [1]:
import os
import sys
import glob

import csv
import json

import unicodedata
import re

import urllib.parse

import numpy as np
import matplotlib.pyplot as plt

## Lookups

Obtained from gps-speedsurfing.com

In [2]:
CACHE_DIR = 'cache'
DATA_FOLDER = 'data'
SESSIONS_DIR = 'eventsessions'

GP3S_DEVICES_CSV = 'gp3s-devices.csv'

GPS_BRAND = 'Brand'
GPS_SERIES = 'Series'
GPS_MODEL = 'Model'
GPS_VARIANT = 'Variant'

KPH_TO_KNOTS = 1000 / 1852
KNOTS_TO_KPH = 1852 / 1000

MIN_SPEED_KNOTS = 0

SHOW_BOX_CHARTS = False

SITES = {
    'gps-speed': 'https://www.gps-speedsurfing.com/default.aspx?mnu=events',
    'gps-foil': 'https://www.gps-foilsurfing.com/default.aspx?mnu=events',
    'gps-wing': 'https://www.gps-wingfoiling.com/default.aspx?mnu=events',
    'gps-kite': 'https://www.gps-kitesurfing.com/default.aspx?mnu=events',
    'gps-ice': 'https://www.gps-icesailing.com/default.aspx?mnu=events'
}

speedTypes = {
    "speed_avg": "Average speed",
    "speed_100": "100 m run",
    "speed_250": "250 m run",
    "speed_500": "500 m run",
    "speed_halfhour": "1/2 hour speed",
    "speed_hour": "1 hour avg speed",
    "speed_24hour": "24 hour",
    "speed_mile": "Nautical mile",
    "speed_10sec": "Max. 10 sec. run",
    "speed_2sec": "Max. 2 sec.",
    "speed_alpha_racing": "Alpha racing"
}

## Load Events

In [3]:
def getUid(sessionUrl):
    """Get user ID from session URL"""

    parsedUrl = urllib.parse.urlparse(sessionUrl)
    uid = urllib.parse.parse_qs(parsedUrl.query)["uid"][0]
            
    return int(uid)

In [4]:
def loadEventSessions(apiName, eventId):
    """Load event sessions"""
    
    sessions = {}

    filename = os.path.join(projdir, 'cache', apiName, 'eventsessions', str(eventId) + '.json')
    with open(filename) as f:
        try:
            sessions = json.load(f)
        except Exception:
            print('Issue parsing {}'.format(filename))
            raise

    return sessions

In [5]:
def slugify(value):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')


def loadGpsTypes():
    '''Load GPS types into dictionary'''

    gpsTypes = {}

    filename = os.path.join(projdir, DATA_FOLDER, GP3S_DEVICES_CSV)   
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for gpsType in reader:
            gpsType['slug'] = slugify('{} {} {}'.format(gpsType[GPS_BRAND], gpsType[GPS_MODEL] or gpsType[GPS_SERIES], gpsType[GPS_VARIANT]))
            gpsTypes[gpsType['GP3S']] = gpsType

    return gpsTypes


def loadSessions(site):
    '''Load de-duplicated sessions into dictionary'''

    sessions = {}

    for filename in glob.glob(os.path.join(projdir, CACHE_DIR, site, SESSIONS_DIR, '*.json')):
        with open(filename) as f:
            eventSessions = json.load(f)

            for eventSession in eventSessions:
                sessions[eventSession['session_id']] = eventSession

    return sessions


def getGpsType(session, gpsTypes):
    '''Get the cleansed GPS type for the session'''

    sessionGpsType = session['sessiongpstype'].strip()
    
    if sessionGpsType in gpsTypes:
        gpsType = gpsTypes[sessionGpsType]
    else:
        print(f'Unrecognised GPS type - {gpsType}')
        gpsType = gpsTypes['unknown']

    return gpsType


def groupSessions(sessions, gpsTypes):
    '''Group sessions based on the device type'''

    groups = {}

    for sessionId, session in sessions.items():
        gpsType = getGpsType(session, gpsTypes)
        slug = gpsType['slug']

        if slug not in groups:
            groups[slug] = {}

        groups[slug][sessionId] = session

    return groups


def drawPlot(diffs, slug):
    '''Draw box plot'''

    fig = plt.figure(figsize=(10, 7))
    
    plt.boxplot(diffs)
    plt.xticks([1], [slug])
    plt.show()


def analyseGroups(groups):
    '''Group sessions based on the device type'''

    stats = {}

    for slug in sorted(groups.keys()):
        diffs = []
        sessions = []
        for session in groups[slug].values():
            if session['speed_2sec'] > MIN_SPEED_KNOTS * KNOTS_TO_KPH and session['speed_100'] > 0:
                spd2s = session['speed_2sec'] * KPH_TO_KNOTS
                spd10s = session['speed_10sec'] * KPH_TO_KNOTS
                spd100m = session['speed_100'] * KPH_TO_KNOTS
                dur100m = 3600 / (spd100m * 1852 / 100)

                # Interpolation is really bad when duration of 100 m is close to 10 seconds
                if abs(10 - dur100m) > 2:
                    # Interpolation will use the 100 m and 10 secs speeds
                    xCoords = [dur100m, 10] 
                    yCoords = [spd100m, spd10s] 
    
                    # Determine the coefficients of the slope using a 1st order polynomial
                    coefficients = np.polyfit(xCoords, yCoords, 1)

                    # Calculate plausible 2 secs via interpolation (if slope is in the right direction)
                    #   See https://www.gps-foilsurfing.com/?mnu=user&val=414948&uid=19409
                    if coefficients[0] < 0:
                        y = coefficients[1] + 2 * coefficients[0]
                    else:
                        y = spd10s
                else:
                    y = spd10s
    
                # Compare to the actual 2s
                diff = spd2s - y

                # TODO - consider the fact that a 10s 100m has more scope for a fast 2s than a 5s 100m
                
                diffs.append(diff)
                sessions.append(session)

        if len(diffs) > 0:
            # Useful debug info
            quantiles = np.quantile(diffs, [0, 0.25, 0.5, 0.75, 1])
            stats[slug] = quantiles    
            upperFence = quantiles[2] + (quantiles[3] - quantiles[1]) * 1.5
    
            if slug.startswith('motion'):
                if SHOW_BOX_CHARTS:
                    print('{} sessions plotted for {}'.format(len(diffs), slug))
                    drawPlot(diffs, slug)

                #print('{} {} sessions: quantiles = {:0.1f} {:0.1f} {:0.1f} {:0.1f} {:0.1f}, upper fence = {:0.1f}'.format(len(groups[slug]), slug,
                    #quantiles[0], quantiles[1], quantiles[2], quantiles[3], quantiles[4], upperFence))

                for i, diff in enumerate(diffs):
                    # 5 shows massive spikes, 3 shows large spikes, 2 shows medium spikes
                    if diff > 2: #upperFence:
                        
                        message = ' - '.join((sessions[i]['sessiongpstype'], sessions[i]['sessionurl'], str(round(diff, 1)),
                                              sessions[i]['session_date'], sessions[i]['username']))
                        print(message)

    return stats


if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

    gpsTypes = loadGpsTypes()

    for site in SITES:
        sessions = loadSessions(site)

        print('{} sessions loaded from {}'.format(len(sessions), site))

        groups = groupSessions(sessions, gpsTypes)

        stats = analyseGroups(groups)

        print()
    
print('All done!')

36123 sessions loaded from gps-speed
Motion - https://www.gps-speedsurfing.com/?mnu=user&val=436607&uid=25893 - 2.4 - 4/15/2024 12:00:00 AM - Robin Gosselaar
Motion - https://www.gps-speedsurfing.com/?mnu=user&val=436608&uid=25679 - 2.5 - 4/15/2024 12:00:00 AM - Boy van der Veer
Motion - https://www.gps-speedsurfing.com/?mnu=user&val=389454&uid=21874 - 2.3 - 7/31/2022 12:00:00 AM - Lloyd Fierloos
Motion - https://www.gps-speedsurfing.com/?mnu=user&val=398702&uid=25991 - 2.3 - 11/19/2022 12:00:00 AM - Andreas Kraxner

3782 sessions loaded from gps-foil
Motion - https://www.gps-foilsurfing.com/?mnu=user&val=395371&uid=679 - 3.7 - 10/6/2022 12:00:00 AM - Mark Newman
Motion - https://www.gps-foilsurfing.com/?mnu=user&val=397636&uid=24571 - 2.2 - 10/26/2022 12:00:00 AM - John John Vanderick

1421 sessions loaded from gps-wing

41 sessions loaded from gps-kite

396 sessions loaded from gps-ice

All done!
